# Advent of Code 2021
## [Day 14: Extended Polymerization](https://adventofcode.com/2021/day/14)

#### Load Data

In [1]:
import numpy as np
from collections import defaultdict

In [2]:
import aocd
input_data = aocd.get_data(year=2021, day=14).split('\n')

In [3]:
test_data = [
    'NNCB',
    '',
    'CH -> B',
    'HH -> N',
    'CB -> H',
    'NH -> C',
    'HB -> C',
    'HC -> B',
    'HN -> C',
    'NN -> C',
    'BH -> H',
    'NC -> B',
    'NB -> B',
    'BN -> B',
    'BB -> N',
    'BC -> B',
    'CC -> N',
    'CN -> C'
]
test_template = test_data[0]
test_rules = test_data[2:]

In [4]:
def parse_data(data):
    template = data[0]
    rules = dict(line.split(' -> ') for line in data[2:])
    return template, rules

template, rules = parse_data(test_data)
template, rules

('NNCB',
 {'CH': 'B',
  'HH': 'N',
  'CB': 'H',
  'NH': 'C',
  'HB': 'C',
  'HC': 'B',
  'HN': 'C',
  'NN': 'C',
  'BH': 'H',
  'NC': 'B',
  'NB': 'B',
  'BN': 'B',
  'BB': 'N',
  'BC': 'B',
  'CC': 'N',
  'CN': 'C'})

In [5]:
def apply_rules(template, rules):
    result = template[0]
    for i in range(len(template)-1):
        pair = template[i:i+2]
        if pair in rules:
            result += rules[pair]
        result += pair[1]
    return result

apply_rules(*parse_data(test_data))

'NCNBCHB'

In [6]:
def iterate_rules(template, rules, n):
    result = template
    for i in range(n):
        result = apply_rules(result, rules)
        # print(i+1, result)
    return result
result = iterate_rules(*parse_data(test_data), 10)
result = np.array(list(result))

In [7]:
for element in np.unique(result):
    print(f"{element}: {np.sum(result == element)}")

B: 1749
C: 298
H: 161
N: 865


In [8]:
1749 - 161

1588

In [9]:
element_counts = [np.sum(result == element) for element in np.unique(result)]
np.ptp(element_counts)

1588

#### Part 1 Answer
Apply 10 steps of pair insertion to the polymer template and find the most and least common elements in the result. 
**What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?**

In [10]:
result = iterate_rules(*parse_data(input_data), 10)
result = np.array(list(result))

for element in np.unique(result):
    print(f"{element}: {np.sum(result == element)}")

B: 1811
C: 906
F: 1991
H: 1597
K: 946
N: 1239
O: 4461
P: 2617
S: 1645
V: 2244


In [11]:
element_counts = [np.sum(result == element) for element in np.unique(result)]
np.ptp(element_counts)

3555

### Part 2

The above algorithm will not scale to 40 iterations.

In [12]:
# iterate_rules(*parse_data(test_data), 40)

In [13]:
def get_bigrams(template):
    template = template + '$'
    bigrams = defaultdict(lambda: 0)
    for i in range(len(template)-1):
        pair = str(template[i:i+2])
        bigrams[pair] += 1
    return bigrams

template, rules = parse_data(test_data)
bigrams = get_bigrams(template)
bigrams

defaultdict(<function __main__.get_bigrams.<locals>.<lambda>()>,
            {'NN': 1, 'NC': 1, 'CB': 1, 'B$': 1})

In [14]:
def apply_rules_to_bigrams(bigrams, rules):
    next_step = bigrams.copy()
    for target, insertion in rules.items():
        num_matches = bigrams[target]
        next_step[target] -= num_matches
        result1 = target[0] + insertion
        next_step[result1] += num_matches
        result2 = insertion + target[1]
        next_step[result2] += num_matches
        # print(target, insertion, result1, result2, num_matches)
    return next_step
        
apply_rules_to_bigrams(bigrams, rules)

defaultdict(<function __main__.get_bigrams.<locals>.<lambda>()>,
            {'NN': 0,
             'NC': 1,
             'CB': 0,
             'B$': 1,
             'CH': 1,
             'BH': 0,
             'HH': 0,
             'HN': 0,
             'NH': 0,
             'HB': 1,
             'HC': 0,
             'BC': 1,
             'CN': 1,
             'NB': 1,
             'BB': 0,
             'BN': 0,
             'CC': 0})

In [15]:
def iterate_rules_b(template, rules, n):
    bigrams = get_bigrams(template)
    for i in range(n):
        bigrams = apply_rules_to_bigrams(bigrams, rules)
    return bigrams
result = iterate_rules_b(*parse_data(test_data), 10)
result

defaultdict(<function __main__.get_bigrams.<locals>.<lambda>()>,
            {'NN': 0,
             'NC': 42,
             'CB': 115,
             'B$': 1,
             'CH': 21,
             'BH': 81,
             'HH': 32,
             'HN': 27,
             'NH': 27,
             'HB': 26,
             'HC': 76,
             'BC': 120,
             'CN': 102,
             'NB': 796,
             'BB': 812,
             'BN': 735,
             'CC': 60})

In [16]:
def count_elements(bigrams):
    elements = defaultdict(lambda: 0)
    for key, val in bigrams.items():
        elements[key[0]] += val
    return elements

count_elements(result)

defaultdict(<function __main__.count_elements.<locals>.<lambda>()>,
            {'N': 865, 'C': 298, 'B': 1749, 'H': 161})

In [17]:
np.ptp([val for key, val in count_elements(result).items()])

1588

#### Part 2 Answer
Apply 40 steps of pair insertion to the polymer template and find the most and least common elements in the result.  
**What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?**

In [18]:
result = iterate_rules_b(*parse_data(input_data), 40)
np.ptp([val for key, val in count_elements(result).items()])

4439442043739